<img SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">

# Caching

*O.N. Ebbens, Artesia, 2021*

Groundwater flow models are often data-intensive. Execution times can be shortened significantly by caching data. This notebooks explains how this caching is implemented in `nlmod`. The first three chapters explain how to use the caching in nlmod. The last chapter contains more technical details on the implementation and limitations of caching in nlmod.

### Contents<a name="TOC"></a>
1. [Cache directory](#cachedir)
2. [Caching in nlmod](#cachingnlmod)
3. [Checking the cache](#3)
4. [Apply caching to a function](#4)
5. [Technicalities and discussion](#5)

In [1]:
import matplotlib.pyplot as plt
import flopy
import os
import geopandas as gpd
import xarray as xr
import numpy as np
import logging

import nlmod

# toon informatie bij het aanroepen van functies
logging.basicConfig(level=logging.INFO)
print(f'nlmod version: {nlmod.__version__}')

nlmod version: 0.2.5b


### [1. Cache directory](#TOC)<a name="cachedir"></a>

When you create a model you usually start by assigning a model workspace. This is a directory where model data is stored. The `nlmod.util.get_model_dirs()` function can be used to create a file structure in two steps:
1. The model workspace directory is created if it does not exist yet. 
2. Two subdirectories are created: 'figure' and 'cache'. 

Calling the function below we create the `figdir` and `cachedir` variables with the paths of the subdirectories. In this notebook we will use this `cachedir` to write and read cached data. It is possible to define your own cache directory.

In [2]:
model_ws = 'model5'

# Model directories
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)

print(model_ws)
print(figdir)
print(cachedir)

model5
model5\figure
model5\cache


### [2. Caching in nlmod](#TOC)<a name="cachingnlmod"></a>

In `nlmod` you can use the `get_combined_layer_models` function to obtain a layer model based on `regis`.

In [3]:
layer_model = nlmod.read.regis.get_combined_layer_models(extent=[95000.0, 105000.0, 494000.0, 500000.0],
                                                         use_geotop=False)

As you may notice, this function takes some time to complete because the data is downloaded and projected on the desired model grid. Everytime you run this function you have to wait for the process to finish which results in an unhealthy number of coffee breaks. This is why we use caching. To store our cache we use netCDF files. The `layer_model` variable is an `xarray.Dataset`. You can read/write an `xarray.Dataset` to/from a NetCDF file using the code below.

In [4]:
# write netcdf with layer model data
layer_model.to_netcdf(os.path.join(cachedir, 'layer_test.nc'))

In [5]:
# read netcdf with layer model data
layer_model = xr.open_dataset(os.path.join(cachedir, 'layer_test.nc'), mask_and_scale=False)

Reading and writing netcdf files is the main principle behind caching in `nlmod`. We write the `layer_model` to a NetCDF file when we call the `get_combined_layer_models` function for the first time. The next time we call the function we can read the cached NetCDF file instead. This reduces exuction time signficantly. You can simply use this caching abilities by specifying a `cachedir` and a `cachename` in the function call.

In [6]:
layer_model = nlmod.read.regis.get_combined_layer_models(extent=[95000.0, 105000.0, 494000.0, 500000.0],
                                                         use_geotop=False,
                                                         cachedir=cachedir, 
                                                         cachename='combined_layer_ds.nc')

INFO:nlmod.cache:caching data -> combined_layer_ds.nc


### caching steps<a name="steps"></a>

The netCDF caching is applied to a number of functions in nlmod that have an xarray dataset as output. When you call these functions using the `cachedir` and `cachename` arguments the following steps are taken:
1. See if there is a netCDF file with the specified cachename in the specified cache directory. If the file exists go to step 2, otherwise go to step 3.
2. Read the netCDF file and return as an xarray dataset if:
    1. The cached dataset was created using the same function arguments as the current function call. 
    2. The module where the function is defined has not been changed after the cache was created.
3. Run the function to obtain an xarray dataset. Save this dataset as a netCDF file, using the specified cachename and cache directory, for next time. Also return the dataset.

This is the flowchart of an ordinary function call: <br>
<div>
<img src="img/ordinary_function_call_v2.png" width="400"/>
</div>

This is the flowchart of a function call using the caching from nlmod: <br>
<div>
<img src="img/cache_function_call_v2.PNG" width="800"/>
</div>

### caching functions

The following functions use the caching as described above:
- nlmod.read.regis.get_combined_layer_models
- nlmod.read.regis.get_regis
- nlmod.read.rws.get_surface_water
- nlmod.read.rws.get_northsea
- nlmod.read.knmi.get_recharge
- nlmod.read.jarkus.get_bathymetry
- nlmod.read.geotop.get_geotop
- nlmod.read.ahn.get_ahn

### [3. Checking the cache](#TOC)<a name="3"></a>
One of the steps in the caching process ([step 2A](#steps)) is to check if the cache was created using the same function arguments as the current function call. This check has some limitations:
- Only function arguments with certain types are checked. These types include: int, float, bool, str, bytes, list, tuple, dict, numpy.ndarray, xarray.DataArray and xarray.Dataset. If a function argument has a different type the cache is never used. In time more types can be added to the checks.
- If one of the function arguments is an xarray Dataset the check is somewhat different. For a dataset we only check if it has identical dimensions and coordinates as the cached netcdf file. There is no check if the variables in the dataset are identical.
- It is not possible to cache the results of a function with more than one xarray Dataset as an argument. This is due to the difference in checking datasets. If more than one xarray dataset is given the cache decoraters raises a TypeError.
- If one of the function arguments is a filepath of type str we only check if the cached filepath is the same as the current filepath. We do not check if any changes were made to the file after the cache was created.

You can test how the caching works in different situation by running the function below a few times with different function arguments. The logs provide some information about using the cache or not.

In [7]:
# layer model
layer_model = nlmod.read.regis.get_combined_layer_models(extent=[95000.0, 105000.0, 494000.0, 500000.0],
                                                         use_geotop=False,
                                                         cachename='combined_layer_ds.nc',
                                                         cachedir=cachedir)
layer_model

INFO:nlmod.cache:using cached data -> combined_layer_ds.nc


<xarray.Dataset>
Dimensions:  (layer: 132, y: 60, x: 100)
Coordinates:
  * x        (x) float64 9.505e+04 9.515e+04 9.525e+04 ... 1.048e+05 1.05e+05
  * y        (y) float64 4.94e+05 4.942e+05 4.942e+05 ... 4.998e+05 5e+05
  * layer    (layer) object 'mv' 'HLc' 'BXz1' 'BXSCk1' ... 'GUq' 'VAc' 'AKc'
Data variables:
    top      (layer, y, x) float32 ...
    botm     (layer, y, x) float32 ...
    kh       (layer, y, x) float32 ...
    kv       (layer, y, x) float32 ...
Attributes: (12/40)
    references:                    https://www.dinoloket.nl/regis-ii-het-hydr...
    Conventions:                   CF-1.7
    creator_url:                   https://www.dinoloket.nl
    keywords_vocabulary:           NASA/GCMD Earth Science Keywords. Version 6.0
    acknowledgment:                https://www.dinoloket.nl
    project:                       REGIS v02r2
    ...                            ...
    geospatial_lon_resolution:     0.0014573401
    geospatial_vertical_min:       -1235.92
    geospatial_vertical_max:       322.75
    geospatial_vertical_units:     m-NAP
    geospatial_vertical_positive:  up
    extent:                        [ 95000. 105000. 494000. 500000.]

### clearing the cache

Sometimes you want to get rid of all the cached files to free disk space or to support your minimalistic lifestyle. You can use the `clear_cache` function to clear all cached files in a specific cache directory.

In [8]:
#nlmod.cache.clear_cache(cachedir)

### [4. Apply caching to a function](#TOC)<a name="4"></a>
We have a simple function that takes some input and returns an xarray Dataset. We've put this function in the cache_example.py so we can import it in this notebook. In the module we used the `nlmod.cache.cache_netcdf` decorator on the function.

In [9]:
from cache_example import func_to_create_a_dataset

Now we can call the function to create a simple Dataset.

In [10]:
func_to_create_a_dataset(10)

<xarray.Dataset>
Dimensions:  (x: 100)
Coordinates:
  * x        (x) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 96.0 97.0 98.0 99.0 100.0
Data variables:
    test     (x) float64 10.0 10.0 10.0 10.0 10.0 ... 10.0 10.0 10.0 10.0 10.0

We can also call this function using the `cachedir` and `cachename` arguments, even though we haven't defined these arguments in our function definition. The arguments were added by the decorator. 

In [11]:
func_to_create_a_dataset(10, cachedir=cachedir, cachename='example')

INFO:nlmod.cache:caching data -> example.nc


<xarray.Dataset>
Dimensions:  (x: 100)
Coordinates:
  * x        (x) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 96.0 97.0 98.0 99.0 100.0
Data variables:
    test     (x) float64 10.0 10.0 10.0 10.0 10.0 ... 10.0 10.0 10.0 10.0 10.0

Now the caching logic described in this notebook is applied to the function call. Next time we call this function it returns the cached dataset

In [12]:
func_to_create_a_dataset(10, cachedir=cachedir, cachename='example')

INFO:nlmod.cache:using cached data -> example.nc


<xarray.Dataset>
Dimensions:  (x: 100)
Coordinates:
  * x        (x) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 96.0 97.0 98.0 99.0 100.0
Data variables:
    test     (x) float64 ...

Or it creates a new cached dataset if our function arguments have been changed:

In [13]:
func_to_create_a_dataset(11, cachedir=cachedir, cachename='example')

INFO:nlmod.cache:cache was created using different function argument values, do not use cached data
INFO:nlmod.cache:caching data -> example.nc


<xarray.Dataset>
Dimensions:  (x: 100)
Coordinates:
  * x        (x) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 96.0 97.0 98.0 99.0 100.0
Data variables:
    test     (x) float64 11.0 11.0 11.0 11.0 11.0 ... 11.0 11.0 11.0 11.0 11.0

When we decorate a function the cachedir and cachename arguments are also added to the docstring as you can see below:

In [14]:
# show that the arguments cachedir and cachename are added to the docstring
func_to_create_a_dataset?

Signature: func_to_create_a_dataset(number, cachedir=None, cachename=None)
Docstring:
create a dataarray as an example for the caching method.

Parameters
----------
number : int, float
    values in data array
cachedir : str or None, optional
    directory to save cache. If None no cache is used. Default is None.
cachename : str or None, optional
    filename of netcdf cache. If None no cache is used. Default is None.

Returns
-------
da : xr.DataArray
File:      c:\users\oebbe\02_python\nlmod\docs\examples\cache_example.py
Type:      function


### [5. Technicalities and discussion](#TOC)<a name="5"></a>

In nlmod we use a specific caching method called [memoization](https://en.wikipedia.org/wiki/Memoization). The memoization is implemented in the `nlmod` caching module. The `cache_netcdf` decorator function handles most of the magic for caching netcdf files. When the cache is created all function arguments are stored in a dictionary and saved (pickled) as a .pklz file. The check on function arguments (step 2A) is done by reading the pickle and comparing the output with the arguments of the current function call.


#### Properties
1. All function arguments are pickled and saved together with the netcdf file. If the function arguments use a lot of memory this process can be become slow. This should be taken into account when you decide to use caching.
2. Function arguments that cannot be pickled using the `pickle` module raise an error in the caching process.
3. A function with mutable function arguments that are modified during function execution should not be used in caching. It can be used but the cache will never be used. The check on function arguments will always be False since the original function arguments are compared with the modified function argument.
4. If one of the function arguments is an xarray Dataset we only check if the dataset has the same dimensions and coordinates as the cached netcdf file. There is no check on the variables (DataArrays) in the dataset because it would simply take too much time to check all the variables in the dataset. Also, most of the time it is not necesary to check all the variables as they are not used to create the cached file. There is one example where a variable from the dataset is used to create the cached file. The `nlmod.read.jarkus.bathymetry_to_model_dataset` uses the 'Northsea' DataArray to create a bathymetry dataset. When we access the 'Northsea' DataArray using `model_ds['Northsea']` in the `bathymetry_to_model_dataset` function there would be no check if the 'Northsea' DataArray that was used to create the cache is the same as the 'Northsea' DataArray in the current function call. The current solution for this is to make the 'Northsea' DataArray a separate function argument in the `bathymetry_to_model_dataset` function. This makes it also more clear which data is used in the function.
5. There is a check to see if the module where the function is defined has been changed since the cache was created. This helps not to use the cache when changes are made to the function. Unfortunately when the function uses other functions from different modules these other modules are not checked for recent changes.
6. The `cache_netcdf` decorator uses `functools.wraps` and some home made magic to add properties, such as the name and the docstring, of the original function to the decorated function. This assumes that the original function has a docstring with a "Returns" heading. If this is not the case the docstring is not modified.

#### storing cache on disk

Many memoization methods use a hash of the function arguments as the filename. Thus creating multiple files for different function calls. The memoization in `nlmod` uses a user-defined filename (`cachename`) to store the cache. If the function is called with different arguments the previous cached file is overwritten. By not creating a new file for every unique set of function arguments we reduce the number of files and therefore the memory size on the disk. By saving the function output as netCDF file it is also possible to read the file seperately from the caching process. While this is not something you would often do it can help when debugging. 